In [0]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My Drive/Project ABBYY/gicrapi

/content/drive/My Drive/Project ABBYY/gicrapi


In [3]:
!ls

2.ipynb		      demo1.sh			  parser.ipynb
data.csv	      gender.ipynb		  text_classification.ipynb
data_normal_form.csv  gender_with_file_xml.ipynb  Untitled0.ipynb
demo1.py	      gicr			  vk_part.tnt.xml


In [0]:
from lxml import etree
import numpy as np
import pandas as pd
import re

In [0]:
#Находит значение атрибута
def parse_attributes(attribute, line):
    result = re.search(r'{}=.(\w+).'.format(attribute), line)
    if result:
        return result.group(1)
    #Если информации нет, возвращает строку 'NA'
    return 'NA'

#Применяет ругулярку к строке
def parse_text(reg, line):
    result = re.search(reg, line)
    if result:
        return result.group(1)
    #<o composite="true"> не обробатывается (я не знаю что это) и еще что то непонятное
    #МОЖЕТ ВЕРНУТЬ ПУСТУЮ СТРОКУ
    return ""
        
def parse_article(file):
    df = pd.DataFrame()
    
    attributes_list = ["source", "genrei", "gender", "birth", "nickname",
                      "id", "loc", "month", "rule", "year"]
    
    attributes_dict = {}
    for attr in attributes_list:
        attributes_dict[attr] = []
    attributes_dict["text"] = []
    attributes_dict["words"] = []
    attributes_dict["infinitives"] = []
    attributes_dict["specification"] = []
    
    text = ""
    words = []
    infinitives = []
    specification = []
    
    for line in file:
        if line.startswith('</text>'):
            
            attributes_dict["text"].append(text)
            attributes_dict["words"].append(words)
            attributes_dict["infinitives"].append(infinitives)
            attributes_dict["specification"].append(specification)
            
            words = []
            specification = []
            infinitives = []
            text = ""
            
        elif line.startswith('<text'):
            
            for attr in attributes_list:
                if attr == "loc":
                    attributes_dict[attr].append(re.findall(r'loc=.(\w+).', line))
                else:
                    attributes_dict[attr].append(parse_attributes(attr, line))
        else:
            
            words.append(parse_text(r'(\S+)\s+\S+\s+\S+', line))
            infinitives.append(parse_text(r'\S+\s+\S+\s+(\S+)', line))
            specification.append(parse_text(r'\S+\s+(\S+)\s+\S+', line))
            text += line + "\n"
            
    df = df.from_dict(attributes_dict)
    return df
        

In [0]:
def parse_file():
    path_to_xml = "vk_part.tnt.xml"
    with open(path_to_xml) as file:
        return parse_article(file)

In [0]:
df = parse_file()


In [0]:
column = ['gender', 'infinitives', 'specification']
data = df[column]

In [9]:
dt = data[data.gender != "NA"]
dt.shape

(2165, 3)

In [0]:
data_F = data[data.gender == "F"]
data_M = data[data.gender == "M"]

In [0]:
sh = list(range(data_F.shape[0]))
np.random.shuffle(sh)
index = sh[:data_M.shape[0]]
data_F = data_F.iloc[index]
dt = data_M.append(data_F, ignore_index=True)


In [0]:
X_F = data_F['infinitives']
X_M = data_M['infinitives']

In [0]:
def bag_words(data):
  bag = {}
  for sentence in data['infinitives']:
    for word in sentence:
      if word != "":
        if bag.get(word) != None:
            bag[word] = bag[word] + 1
        else:
            bag.update({word : 1})
  return bag

In [0]:
bag_F = bag_words(data_F)
bag_M = bag_words(data_M)

In [0]:
from collections import Counter
cbag = Counter(bag_F) + Counter(bag_M)
bag = dict(cbag)

In [0]:
def normal(bag):
  count_word = sum(bag.values())
  return {key : bag[key] / count_word for key in bag.keys()}

In [0]:
def keyness(bag, n, dic_normal):
  return {key : (bag[key] + n)/(dic_normal[key] + n) for key in bag.keys()}

In [0]:
n = 0.03
dic_normal = normal(bag)
bag_F_normal = normal(bag_F)
bag_M_normal = normal(bag_M)


keyness_F = keyness(bag_F_normal, n, dic_normal)
keyness_M = keyness(bag_M_normal, n, dic_normal)

In [0]:
def keyword(keyness, c):
  kw = {}
  for key in keyness.keys():
    if keyness[key] > c:
      kw.update({key: keyness[key]})
  return kw

In [0]:
c = 0.
keyword_F = keyword(keyness_F, c)
keyword_M = keyword(keyness_M, c)

#Used features

In [0]:
X_oh = pd.DataFrame(columns=['F', 'M'])
X = dt['infinitives']
F = 0.
M = 0.
n = 0.
for sentence in X:
  F = 0.
  M = 0.
  for word in sentence:
    if keyness_F.get(word) != None:
      a = keyness_F[word]
    else:
      a = 0.
    F = F + a
    if keyness_M.get(word) != None:
      b = keyness_M[word]
    else:
      b = 0.
    M = M + b
  X_oh = X_oh.append({'F': F, 'M': M}, ignore_index=True)
 

In [23]:
X_oh

,F,M
0,27.271507,26.620052
1,10.054866,11.924829
2,87.386670,105.480588
3,7.968612,10.045533
4,6.053988,7.926057
5,10.986035,17.025972
6,5.974880,12.055110
7,5.974880,9.039170
8,8.002273,15.020837
9,4.996028,5.005559


In [24]:
X_oh.shape

(1442, 2)

In [0]:
y = dt['gender']
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X_oh, y, test_size=0.3, 
                                     random_state=0)

In [28]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [30]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           F       0.99      0.92      0.95       223
           M       0.92      0.99      0.95       210

   micro avg       0.95      0.95      0.95       433
   macro avg       0.95      0.95      0.95       433
weighted avg       0.96      0.95      0.95       433



In [31]:
print (accuracy_score(y_test, y_pred))

0.953810623556582
